In [ ]:
!pip install torch lerobot

In [ ]:
!pip install lerobot.data

In [ ]:
!pip install decord

In [ ]:
!pip install transformers num2words

## Training

In [ ]:
# Add parent directory to path to import train module
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from train_smolvla import train

print("\nStarting training process...")
print("Note: We'll use a public dataset for training as our sample is too small")

# Create a temporary directory for training output
train_output_dir = Path('model_output')

print(f"Training output will be saved to: {train_output_dir}")

#train(output_dir=str(train_output_dir), dataset_id="ISdept/piper_arm")
train(output_dir=str(train_output_dir), dataset_id="ISdept/piper_arm", model_id="ISdept/smolvla-piper", resume_from_checkpoint=False)

## Prepare dataset

In [ ]:
!python ./data_processing/prepare_dataset.py

In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import HfApi
import os

#ISdept/piper-pick-place-diff

!hf upload \
  'ISdept/piper-pick-place-depth' \
  /Users/eddyma/DEV/Github/lerobot-piper/src/output \
  --repo-type dataset

## Inference

### Webcam inference

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))

!python webcam_inference.py

### Video Inference - Diffusion

In [ ]:
!python video_inference.py

In [ ]:
!python video_inference_close_loop.py

### Video inference - SmolVLA

In [ ]:
!python video_inference_smolvla.py

In [ ]:
!python video_inference_smolvla_close_loop.py

## Visualize

In [ ]:
import pandas as pd

# Read the parquet file
# Replace 'your_file.parquet' with your actual file path
df = pd.read_parquet('output/meta/episodes/chunk-000/file-000.parquet', engine='pyarrow') 
#df = pd.read_parquet('output/data/chunk-000/file-000.parquet', engine='pyarrow') 
#df = pd.read_parquet('output/meta/tasks.parquet', engine='pyarrow') 


#df = pd.read_parquet('temp/file-000-hf-data.parquet', engine='pyarrow')
#df = pd.read_parquet('temp/file-000-hf-ep.parquet', engine='pyarrow')
#df = pd.read_parquet('temp/tasks-hf.parquet', engine='pyarrow')



# Display the first 5 rows of the DataFrame
df.head(60)



In [ ]:
!python extract_joint_positions.py

import json
import matplotlib.pyplot as plt
import numpy as np

def plot_joint_positions(json_file_path, title, start_frame_index=0):
    """
    Plots joint positions from a JSON file, starting from a specified frame index.
    Handles both inference results format and joint positions format.

    Parameters:
    json_file_path (str): Path to the JSON file.
    title (str): Title for the plot.
    start_frame_index (int): The frame index from which to start plotting. Defaults to 0 (the beginning).
    """
    # Read the JSON file
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    
    # Check if this is inference results format (list of objects with 'result' key)
    # or joint positions format (list of arrays)
    if isinstance(data, list) and len(data) > 0:
        # This is joint positions format (list of arrays)
        # For this format, we'll just plot all data starting from start_frame_index
        if start_frame_index >= len(data):
            print(f"No data found starting from frame index {start_frame_index}.")
            return
        
        # Extract joint positions from start_frame_index onward
        filtered_data = data[start_frame_index:]
        frame_indices = list(range(start_frame_index, start_frame_index + len(filtered_data)))
        print(f"Frames plotted: {len(frame_indices)} (from index {min(frame_indices)} to {max(frame_indices)})")
        
        # Initialize lists for each joint
        joints = [[] for _ in range(7)]  # 6 joints + 1 gripper
        
        # Extract joint positions for each frame in the filtered data
        for action in filtered_data:
            for i in range(7):  # 6 joints + 1 gripper
                joints[i].append(action[i])

    
    # Create the plot
    plt.figure(figsize=(12, 4)) # Slightly larger figure for clarity
    
    # Joint names
    joint_names = ['Joint 1', 'Joint 2', 'Joint 3', 'Joint 4', 'Joint 5', 'Joint 6', 'Gripper']
    
    # Plot each joint with a different color
    for i in range(7):
        plt.plot(frame_indices, joints[i], label=joint_names[i], marker='o', markersize=3, linewidth=1.5)
    
    # Add labels and title
    plt.xlabel('Frame Index')
    plt.ylabel('Joint Position')
    plt.title(f"{title} (Starting from frame {start_frame_index})")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left') # Legend outside plot
    plt.grid(True, alpha=0.3)
    
    # Show the plot
    plt.tight_layout()
    plt.show()

plot_joint_positions('temp/inference_actions.json', 'Predicted Joint Positions Training')
plot_joint_positions('temp/data_20251128_095915_gt.json', 'Ground Truth Joint Positions')
plot_joint_positions('temp/inference_actions_close_loop.json', 'Predicted Joint Positions Closed Loop')

In [ ]:
from pathlib import Path
from lerobot.policies.diffusion.modeling_diffusion import DiffusionPolicy
output_directory = Path("outputs/eval/example_pusht_diffusion")
# Comment out the old pretrained model path
 # pretrained_policy_path = "lerobot/diffusion_pusht"
# Use your newly trained model path instead
pretrained_policy_path = Path("outputs/train/example_pusht_diffusion")
policy = DiffusionPolicy.from_pretrained("ISdept/piper_arm")

print(policy.config)

In [ ]:
# Updated plotting functionality using the new plotting utility
import sys
from pathlib import Path
sys.path.append(str(Path().resolve()))

from plotting_utils import plot_joint_positions

# Plot the data using the improved function that handles both file formats
plot_joint_positions('temp/inference_actions.json', 'Predicted Joint Positions - Episode 1')
plot_joint_positions('temp/metadata_20251113_080958_gt.json', 'Ground Truth Joint Positions')

In [ ]:
!python inspect_local_parquet.py

In [ ]:
!lerobot-train \
  --dataset.repo_id=ISdept/piper_arm \
  --policy.type=diffusion \
  --output_dir=outputs/train/output_diff3 \
  --job_name=pick_and_place \
  --policy.device=cuda \
  --policy.repo_id=ISdept/piper_arm \
  --wandb.enable=false \
  --dataset.revision="main" \
  --dataset.image_transforms.enable=True \
  --policy.use_separate_rgb_encoder_per_camera=True \
  --policy.crop_shape=[400,400] \
  --save_checkpoint=True \
  --save_freq=2000 \
  \
  --steps=25000 \
  --policy.n_obs_steps=10 \
  --policy.horizon=24 \
  --batch_size=3 \
  --num_workers=4

In [ ]:
!lerobot-eval \
  --policy.repo_id="ISdept/piper_arm" \
  --policy.type="diffusion" \
  --policy.device="mps" \
  --env.type="aloha" \
  --eval.n_episodes=10 \
  --output_dir="outputs/inference/piper_arm_eval" \
  --job_name="piper_arm_diffusion_eval"

In [ ]:
!lerobot-dataset-viz \
    --repo-id ISdept/piper_arm \
    --episode-index 020 \
    --root /Users/eddyma/DEV/Github/lerobot-piper/src/output

In [12]:
!lerobot-imgtransform-viz \
  --repo-id=ISdept/piper_arm \
  --output-dir=transform_examples \
  --n-examples=5

usage: lerobot-imgtransform-viz [-h] [--config_path str] [--repo_id str]
                                [--root str] [--episodes str]
                                [--image_transforms str]
                                [--image_transforms.enable str]
                                [--image_transforms.max_num_transforms str]
                                [--image_transforms.random_order str]
                                [--image_transforms.tfs str] [--revision str]
                                [--use_imagenet_stats str]
                                [--video_backend str] [--streaming str]
lerobot-imgtransform-viz: error: unrecognized arguments: --repo-id=ISdept/piper_arm --output-dir=transform_examples --n-examples=5


In [12]:
obs_temporal_window = [ -i * 0.1 for i in range(2) ][::-1]
obs_temporal_window

[-0.1, 0.0]

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))

!python ../convert.py

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata

dataset = LeRobotDataset("ISdept/piper_arm", force_cache_sync=True, revision="main")


In [ ]:
import torch
dataset.hf_dataset

train_dataloader = torch.utils.data.DataLoader(
        dataset,
        num_workers=0,
        batch_size=20,
        shuffle=False,
        pin_memory=True,
        drop_last=True,
    )

idx = 0
for batch in train_dataloader:
    #print(batch)
    print(batch['episode_index'], len(batch['observation.state']))
    print('---', idx)
    idx += 1
    if idx >= 10:
        break

In [ ]:
import torch
weights = torch.exp(-0.3 * torch.arange(8))
print(weights)
weights = weights / weights.sum() # Normalize
weights

r = [ -i * 0.1 for i in range(8) ][::-1]
r